In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
from numpy.random import default_rng
import risktools as rt
from time import time


In [ ]:
T = 10
sims = 10000
dt = 1/252

In [ ]:
sigma = np.ones((int(T/dt), sims)) * 0.2

In [ ]:
sigma[int(T/dt/2):,:] = 0.5

In [ ]:
rt.simOU(sigma=sigma, log_price=False, sims=sims, T=T, dt=dt, c=False).iloc[:,:100].plot(legend=False)

In [ ]:
rt.simOU(sigma=sigma, log_price=False, sims=sims, T=T, dt=dt, c=True).iloc[:,:100].plot(legend=False)

In [ ]:
rt.simOU(sigma=sigma, log_price=False, sims=sims, T=T, dt=dt, c=True).iloc[:,:100].plot(legend=False)

In [ ]:
%%timeit -n1 -r3
rt.simOU(log_price=False, sims=sims, T=T, c=False)

In [ ]:
%%timeit -n1 -r3
rt.simOU(log_price=True, sims=sims, T=T, c=False)

In [ ]:
%%timeit -n1 -r3
rt.simOU(log_price=False, sims=sims, T=T, c=True)

In [ ]:
%%timeit -n1 -r3
rt.simOU(log_price=True, sims=sims, T=T, c=True)

In [ ]:
2.33/3.66

In [ ]:
from numpy.random import default_rng

In [ ]:
rng = default_rng()

In [ ]:
rng.normal()

In [ ]:
# x = rt.simGBM_MV(s0=[5,5], r=0.01, sigma=[0.2,0.2], T=1, dt=1/252, cor=[[1,0],[0,1]], sims=100)
# pd.DataFrame(x[:,:,0]).plot(legend=False)

In [ ]:
import numpy as np
import risktools as rt

In [ ]:
s0 = 5
mu = 4
theta = 2
dt = 1/252
T = 1
sigma = 0.32

sigma = np.ones((252, 1000)) * 0.1
sigma[100:,:] = 0.5

df = rt.simOU(s0, mu, theta, sigma, T=1, dt=1/252, sims=1000, seed=12345, log_price=False, c=False)

df.iloc[152,:].std()/df.iloc[25,:].std()

In [ ]:
df.iloc[:,:100].plot(legend=False)

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
from numpy.random import default_rng
import risktools as rt
from time import time


In [2]:
rt.extensions.csimOUJ

<function extensions.csimOUJ>

In [ ]:
T = 10
sims = 10000
dt = 1/252